for combining the six unicommerce dataset

In [1]:
import pandas as pd
import os

def process_files(file_paths, output_path):
    # Initialize an empty list to hold DataFrames
    dataframes = []

    # Iterate over file paths and load each file
    for i, input_path in enumerate(file_paths):
        # Validate the file path
        if not os.path.exists(input_path):
            print(f"Error: File not found at {input_path}")
            return
        
        # Load the data based on file type (CSV or Excel)
        try:
            if input_path.endswith('.csv'):
                df = pd.read_csv(input_path, low_memory=False)
            else:
                df = pd.read_excel(input_path)
        except Exception as e:
            print(f"Error loading file {input_path}: {e}")
            return

        # Check if the file is among the first three (export courier) or the last three (export reverse)
        if i < 3:  # For the first three files (Export Courier)
            df = df[['AWB No', 'Shipping Package Status']]
        else:  # For the last three files (Export Reverse)
            df = df.rename(columns={'Tracking No': 'AWB No', 'Reverse Pickup Status': 'Shipping Package Status'})
            df = df[['AWB No', 'Shipping Package Status']]
        
        # Add the DataFrame to the list
        dataframes.append(df)

    # Combine all DataFrames into a single DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Save the combined DataFrame to the specified output path
    try:
        if output_path.endswith('.csv'):
            combined_df.to_csv(output_path, index=False)
        else:
            combined_df.to_excel(output_path, index=False)
        print(f"Combined file saved to {output_path}")
    except Exception as e:
        print(f"Error saving the file: {e}")

if __name__ == "__main__":
    # Provide paths for six input files
    file_paths = [
        input("Enter the path for export courier file 1 (CSV or XLSX): "),
        input("Enter the path for export courier file 2 (CSV or XLSX): "),
        input("Enter the path for export courier file 3 (CSV or XLSX): "),
        input("Enter the path for export reverse file 4 (CSV or XLSX): "),
        input("Enter the path for export reverse file 5 (CSV or XLSX): "),
        input("Enter the path for export reverse file 6 (CSV or XLSX): ")
    ]
    
    # Get the output file path
    output_path = input("Enter the path for the output file (CSV or XLSX): ")
    
    # Add '.xlsx' as default if no valid extension is provided
    if not output_path.endswith(('.csv', '.xlsx')):
        output_path += '.xlsx'
    
    # Process and combine the files
    process_files(file_paths, output_path)


Cleaned the dataset for comparsion and make it correct format

In [ ]:
import pandas as pd

# Load the provided Excel files
output2_file_path = 'C:/Users/gowth/Downloads/script_returns/output2.xlsx'
returnv2_file_path = 'C:/Users/gowth/Downloads/script_returns/ReturnV2-2024-09-05T08_39_23.144Z.xlsx'

# Load the first sheet of both files
output2_df = pd.read_excel(output2_file_path)
returnv2_df = pd.read_excel(returnv2_file_path)

# Clean the "Tracking No" and "Fwdtracking Id" columns
# Convert both columns to string and ensure proper formatting
output2_df['Tracking No'] = output2_df['Tracking No'].astype(str).apply(lambda x: '{:.0f}'.format(float(x)) if 'E+' in x else x)
returnv2_df['Fwdtracking Id'] = returnv2_df['Fwdtracking Id'].astype(str).apply(lambda x: '{:.0f}'.format(float(x)) if 'E+' in x else x)

# Save the cleaned data back to new Excel files
output2_cleaned_file_path = 'output2_cleaned.xlsx'
returnv2_cleaned_file_path = 'ReturnV2_cleaned.xlsx'

# Write the cleaned data to Excel
output2_df.to_excel(output2_cleaned_file_path, index=False)
returnv2_df.to_excel(returnv2_cleaned_file_path, index=False)


check the matching with fwdtracking id , awb no and status for forcesight tool and unicommerce data 

In [ ]:
import pandas as pd

def process_files(awb_path, returnv2_path, output_matched_path, output_unmatched_path, output1_path, output2_path, output3_path):
    # Load the datasets
    awb_df = pd.read_excel(awb_path)
    returnv2_df = pd.read_excel(returnv2_path)

    # Clean and prepare the data: remove NaN values and strip whitespaces
    returnv2_df.dropna(subset=['Fwdtracking Id'], inplace=True)
    awb_df['Tracking No'] = awb_df['Tracking No'].astype(str).str.strip()
    returnv2_df['Fwdtracking Id'] = returnv2_df['Fwdtracking Id'].astype(str).str.strip()

    # Perform a full outer merge to keep all records, even if they don't match
    merged_df = pd.merge(returnv2_df, awb_df, how='outer', left_on='Fwdtracking Id', right_on='Tracking No')

    # Separate matched and unmatched rows
    matched_df = merged_df[merged_df['Tracking No'].notna() & merged_df['Fwdtracking Id'].notna()]
    unmatched_df = merged_df[merged_df['Tracking No'].isna() | merged_df['Fwdtracking Id'].isna()]

    # Save the results for matched and unmatched rows
    matched_df.to_excel(output_matched_path, index=False)
    unmatched_df.to_excel(output_unmatched_path, index=False)

    # Filtering the data according to the conditions
    output1_df = merged_df[
        (merged_df['Shipping Package Status'] == 'RETURNED') &
        (merged_df['Order Status'].isin(['C RETURN RECEIVED', 'RTO RECEIVED']))
    ][['Channel', 'Order ID', 'Order Item ID', 'Order Date', 'SKU', 'Product ID', 'Fulfillment Channel', 'Return Quantity', 'Return Reason', 'Fwdtracking Id', 'Return Date', 'Order Status', 'Shipping Package Status']]

    output2_df = merged_df[
        ((merged_df['Order Status'] == 'C RETURN RECEIVED') | (merged_df['Order Status'] == 'RTO RECEIVED')) &
        (merged_df['Shipping Package Status'].isin(['COURIER_ALLOCATED', 'CREATED', 'RETURN_ACKNOWLEDGED', 'RETURN_EXPECTED', 'COMPLETE']))
    ][['Channel', 'Order ID', 'Order Item ID', 'Order Date', 'SKU', 'Product ID', 'Fulfillment Channel', 'Return Quantity', 'Return Reason', 'Fwdtracking Id', 'Return Date', 'Order Status', 'Shipping Package Status']]

    # Check if the filtering condition might lead to an empty dataframe
    output3_df = merged_df[
        ((merged_df['Order Status'] == 'RTO TRANSIT') | (merged_df['Order Status'] == 'C RETURN APPROVED')) &
        (merged_df['Shipping Package Status'].isin(['COURIER_ALLOCATED', 'CREATED', 'RETURN_ACKNOWLEDGED', 'RETURN_EXPECTED', 'COMPLETE', 'RETURNED']))
    ][['Channel', 'Order ID', 'Order Item ID', 'Order Date', 'SKU', 'Product ID', 'Fulfillment Channel', 'Return Quantity', 'Return Reason', 'Fwdtracking Id', 'Return Date', 'Order Status', 'Shipping Package Status']]

    # Ensure dataframes are not empty before renaming and saving
    if not output1_df.empty:
        output1_df.rename(columns={'Fwdtracking Id': 'AWB No', 'Order Status': 'Forcesight Tool Status', 'Shipping Package Status': 'Unicommerce Status'}, inplace=True)
        output1_df.to_excel(output1_path, index=False)
    
    if not output2_df.empty:
        output2_df.rename(columns={'Fwdtracking Id': 'AWB No', 'Order Status': 'Forcesight Tool Status', 'Shipping Package Status': 'Unicommerce Status'}, inplace=True)
        output2_df.to_excel(output2_path, index=False)

    if not output3_df.empty:
        output3_df.rename(columns={'Fwdtracking Id': 'AWB No', 'Order Status': 'Forcesight Tool Status', 'Shipping Package Status': 'Unicommerce Status'}, inplace=True)
        output3_df.to_excel(output3_path, index=False)

    # Summary of results
    matched_count = matched_df.shape[0]
    unmatched_count = unmatched_df.shape[0]

    print(f"Matched rows: {matched_count}")
    print(f"Unmatched rows: {unmatched_count}")
    print(f"Output1 rows: {output1_df.shape[0] if not output1_df.empty else 0}")
    print(f"Output2 rows: {output2_df.shape[0] if not output2_df.empty else 0}")
    print(f"Output3 rows: {output3_df.shape[0] if not output3_df.empty else 0}")

if __name__ == "__main__":
    # Input paths from the user
    awb_path = input("Enter the path for the AWB dataset (XLSX): ")
    returnv2_path = input("Enter the path for the ReturnV2 dataset (XLSX): ")
    output_matched_path = input("Enter the path for the output matched file (XLSX): ")
    output_unmatched_path = input("Enter the path for the output unmatched file (XLSX): ")
    output1_path = input("Enter the path for the first filtered output (XLSX): ")
    output2_path = input("Enter the path for the second filtered output (XLSX): ")
    output3_path = input("Enter the path for the third filtered output (XLSX): ")

    # Process the files
    process_files(awb_path, returnv2_path, output_matched_path, output_unmatched_path, output1_path, output2_path, output3_path)
